In [44]:
df = pd.read_csv("ratings.csv")
books = {}
book_list = []
print(df.shape)


(5976479, 3)


In [ ]:
for i in range(0,df.shape[0]):
    book = df.loc[i,'book_id']
    user = df.loc[i,'user_id']
    if book not in books:
        books[book] = []
    books[book].append(user)
for key, val in books.items():
    book_list.append((key,val))

ofile = open('out.csv',"w+")
for i in range(0,len(book_list)-1):
    for j in range(i+1,len(book_list)):
        b1 = book_list[i][0]
        b2 = book_list[j][0]
        common_users = list(set(book_list[i][1]) & set(book_list[j][1]))
        count = len(common_users)
        line = str(b1) + ',' + str(b2) + ',' + str(count) + '\n'
        ofile.write(line)
ofile.close()

In [ ]:
#importing necessary libaries
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
from pyspark import SQLContext
from itertools import islice
from pyspark.sql.functions import col
#creating the context
sqlContext = SQLContext(sc)

#reading the csv file into an RDD
#r = sc.textFile("s3n://goodbooks-10k/data/ratings.csv").map(lambda line: line.split(","))
#b = sc.textFile("s3n://goodbooks-10k/data/books.csv").map(lambda line: line.split(","))
#bt = sc.textFile("s3n://goodbooks-10k/data/book_tags.csv").map(lambda line: line.split(","))
#t = sc.textFile("s3n://goodbooks-10k/data/tags.csv").map(lambda line: line.split(","))
#tr = sc.textFile("s3n://goodbooks-10k/data/to_read.csv").map(lambda line: line.split(","))

#reading the csv file directly into a Pandas dataFrame
# and remove duplicates
b = spark.read.csv("s3n://goodbooks-10k/data/books.csv", escape='"', multiLine=True,
     inferSchema=False, header=True).dropDuplicates()
r = spark.read.csv("s3n://goodbooks-10k/data/ratings.csv", escape='"', multiLine=True,
     inferSchema=False, header=True).dropDuplicates()
bt = spark.read.csv("s3n://goodbooks-10k/data/book_tags.csv", escape='"', multiLine=True,
     inferSchema=False, header=True).dropDuplicates()
t = spark.read.csv("s3n://goodbooks-10k/data/tags.csv", escape='"', multiLine=True,
     inferSchema=False, header=True).dropDuplicates()
tr = spark.read.csv("s3n://goodbooks-10k/data/to_read.csv", escape='"', multiLine=True,
     inferSchema=False, header=True).dropDuplicates()

In [ ]:
# Find books that appear most often in TO-READ shelf
tr_c = tr.groupBy("book_id").count()
tr_b = tr_c.join(b, on="book_id").select("book_id","authors","title",'original_publication_year','average_rating','image_url','small_image_url','count')
tr_b = tr_b.sort(desc("count"))
tr_b.coalesce(1).write.csv('s3n://goodbooks-10k/output/to_read_count',header = 'true')
tr_b.show()

In [ ]:
# Make list of tags for "unfinished"
L = ['left-unfinished\r','series-unfinished\r','unfinished\r','unfinished-abandoned\r',\
    'unfinished-books\r','unfinished-series\r','unifinished\r','incomplete\r','half-read\r']
uf_tags = t.filter(col('tag_name\r').isin(L))
# Count number of times books are marked "unifinished"
from pyspark.sql.types import IntegerType
# join book_tags with list of unfinished tags
uf_books = bt.join(uf_tags,on='tag_id').select('goodreads_book_id','count\r','tag_name\r')
uf_books = uf_books.withColumn('count', regexp_replace('count\r',"\r",""))
uf_books = uf_books.withColumn('count', uf_books['count'].cast(IntegerType()))
# sum all unfinished tags for each book
uf_books= uf_books.groupBy('goodreads_book_id').sum('count')
# Get book details
uf_books = uf_books.join(b, on="goodreads_book_id").select("book_id","authors","title","sum(count)",'image_url','small_image_url')
uf_books = uf_books.sort(desc("sum(count)"))
uf_books.coalesce(1).write.csv('s3n://goodbooks-10k/output/unfinished_books',header = 'true')
uf_books.show(30)

In [ ]:
# Books with highest rating variance
r_variance = r.groupBy('book_id').agg({'rating': 'variance'}).withColumnRenamed("variance(rating)", "variance")
r_variance = r_variance.join(b, on="book_id").select("book_id","authors","title","variance","average_rating")
r_variance = r_variance.sort(desc("variance"))
r_variance.coalesce(1).write.csv('s3n://goodbooks-10k/output/controversial_books',header = 'true')

In [ ]:
# Make list of tags for "boring"
L = ['boring\r','boring-characters\r','just-boring\r','kinda-boring\r',\
    'bored-halfway-through\r']
boring_tags = t.filter(col('tag_name\r').isin(L))
# Count number of times books are marked "boring"
from pyspark.sql.types import IntegerType
# join book_tags with list of unfinished tags
boring = bt.join(boring_tags,on='tag_id').select('goodreads_book_id','count\r','tag_name\r')
boring = boring.withColumn('count', regexp_replace('count\r',"\r",""))
boring = boring.withColumn('count', boring['count'].cast(IntegerType()))
# sum all unfinished tags for each book
boring= boring.groupBy('goodreads_book_id').sum('count')
# Get book details
boring = boring.join(b, on="goodreads_book_id").select("book_id","authors","title","sum(count)",'image_url','small_image_url')
boring = boring.sort(desc("sum(count)"))
boring.coalesce(1).write.csv('s3n://goodbooks-10k/output/boring_books',header = 'true')
boring.show(30)

In [ ]:
# Find most prolific writers
author_works = b.groupBy("authors").agg(count('*'),sum('ratings_count'),avg('average_rating'),sum('ratings_5'),sum('ratings_4'))
author_works= author_works.sort(desc("count(1)"))
author_works.coalesce(1).write.csv('s3n://goodbooks-10k/output/authors',header = 'true')
author_works.show()

In [ ]:
# Most reviewed books and their averaged ratings
r_count = r.groupBy('book_id').count()
r_count = r_count.join(b, on="book_id").select("book_id","authors","title","count","average_rating")
r_count = r_count.sort(desc("count"))
#r_count.show()